# Use Bart to generate story from storyline
References: </br>
article: https://towardsdatascience.com/teaching-bart-to-rap-fine-tuning-hugging-faces-bart-model-41749d38f3ef
</br>
github repo: http://www.github.com/fpaupier/RapLyrics-Scraper


In [2]:
from google.colab import drive
drive.mount('/content/drive')
root_dir = "/content/drive/MyDrive/Colab Notebooks/544_bart/"

Mounted at /content/drive


In [3]:
# This run uses Pytorch Lightening to finetune the model
!pip install -q pytorch-lightning
!pip install -q transformers

     |████████████████████████████████| 524 kB 4.1 MB/s 
     |████████████████████████████████| 329 kB 61.6 MB/s 
     |████████████████████████████████| 132 kB 81.0 MB/s 
     |████████████████████████████████| 829 kB 54.1 MB/s 
     |████████████████████████████████| 596 kB 45.9 MB/s 
     |████████████████████████████████| 1.1 MB 51.6 MB/s 
     |████████████████████████████████| 271 kB 67.9 MB/s 
     |████████████████████████████████| 192 kB 75.7 MB/s 
     |████████████████████████████████| 160 kB 62.1 MB/s 
     |████████████████████████████████| 3.1 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 58.7 MB/s 
     |████████████████████████████████| 3.3 MB 17.6 MB/s 
     |████████████████████████████████| 59 kB 6.9 MB/s 


In [4]:
# imports
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint

import math
import random
import re
import argparse

In [5]:
df = pd.read_csv(root_dir + 'train_title_line_story.csv')
df

,storytitle,storyline,story,titleLineConcat
0,David Drops the Weight,put try realized started weeks,David noticed he had put on a lot of weight re...,David Drops the Weight <EOT> put try realized ...
1,Frustration,tom angry wall tom regret,Tom had a very short temper.One day a guest ma...,Frustration <EOT> tom angry wall tom regret
2,Marcus Buys Khakis,business formal pair perfectly marcus,Marcus needed clothing for a business casual e...,Marcus Buys Khakis <EOT> business formal pair ...
3,Different Opinions,trailer needed much ways truck,Bobby thought Bill should buy a trailer and ha...,Different Opinions <EOT> trailer needed much w...
4,Overcoming shortcomings,pastor tried sing sundays delighted,John was a pastor with a very bad memory.He tr...,Overcoming shortcomings <EOT> pastor tried sin...
...,...,...,...,...
52660,Flavor,flavor tried get recipe recipe,The man liked the flavor.He tried to recreate ...,Flavor <EOT> flavor tried get recipe recipe
52661,After Death,trouble day found told week,"After my friend's dad's funeral, I got in trou...",After Death <EOT> trouble day found told week
52662,Janice breaks her wrist,janice legs working wrist wrist,Janice was out exercising for her big soccer g...,Janice breaks her wrist <EOT> janice legs work...
52663,Jamie marries for love,jamie married man marrying spent,Jamie is an american girl.Jamie wants to get m...,Jamie marries for love <EOT> jamie married man...


In [6]:
train, valid, test = np.split(df, [int(.8*len(df)), int(.9*len(df))])

In [7]:
train

,storytitle,storyline,story,titleLineConcat
0,David Drops the Weight,put try realized started weeks,David noticed he had put on a lot of weight re...,David Drops the Weight <EOT> put try realized ...
1,Frustration,tom angry wall tom regret,Tom had a very short temper.One day a guest ma...,Frustration <EOT> tom angry wall tom regret
2,Marcus Buys Khakis,business formal pair perfectly marcus,Marcus needed clothing for a business casual e...,Marcus Buys Khakis <EOT> business formal pair ...
3,Different Opinions,trailer needed much ways truck,Bobby thought Bill should buy a trailer and ha...,Different Opinions <EOT> trailer needed much w...
4,Overcoming shortcomings,pastor tried sing sundays delighted,John was a pastor with a very bad memory.He tr...,Overcoming shortcomings <EOT> pastor tried sin...
...,...,...,...,...
42127,Long Disney lines,went waited longest leave worth,Last week we went to Disney world.We waited in...,Long Disney lines <EOT> went waited longest le...
42128,Ropes,took went shore rope great,In 2003 my neighbor Dan and I took our sons to...,Ropes <EOT> took went shore rope great
42129,Detention,punished waiting write upset time,The little boy was being punished for spitting...,Detention <EOT> punished waiting write upset time
42130,Forgotten Homework,ryan mom school school teacher,Ryan had forgotten his math assignment at home...,Forgotten Homework <EOT> ryan mom school schoo...


In [8]:
# Create a dataloading module as per the PyTorch Lightning Docs
SOURCE = 'titleLineConcat'
TARGET = 'story'

class SummaryDataModule(pl.LightningDataModule):
  def __init__(self, tokenizer, train, valid, batch_size):
    super().__init__()
    self.tokenizer = tokenizer
    self.train = train
    self.validate = valid
    self.batch_size = batch_size
  
  # Loads and splits the data into training, validation and test sets with a 80/10/10 split
  # def prepare_data(self):
    # self.data = pd.read_csv(self.data_file)[:self.num_examples]
    # self.train, self.validate, self.test = np.split(self.data.sample(frac=1), [int(.8*len(self.data))])

  # encode the sentences using the tokenizer  
  def setup(self, stage):
    self.train = encode_sentences(self.tokenizer, self.train[SOURCE], self.train[TARGET])
    self.validate = encode_sentences(self.tokenizer, self.validate[SOURCE], self.validate[TARGET])
    # self.test = encode_sentences(self.tokenizer, self.test[SOURCE], self.test[TARGET])

  # Load the training, validation and test sets in Pytorch Dataset objects
  def train_dataloader(self):
    dataset = TensorDataset(self.train['input_ids'], self.train['attention_mask'], self.train['labels'])                          
    train_data = DataLoader(dataset, sampler = RandomSampler(dataset), batch_size = self.batch_size)
    return train_data

  def val_dataloader(self):
    dataset = TensorDataset(self.validate['input_ids'], self.validate['attention_mask'], self.validate['labels']) 
    val_data = DataLoader(dataset, batch_size = self.batch_size)                       
    return val_data

  # def test_dataloader(self):
  #   dataset = TensorDataset(self.test['input_ids'], self.test['attention_mask'], self.test['labels']) 
  #   test_data = DataLoader(dataset, batch_size = self.batch_size)                   
  #   return test_data

In [9]:
class LitModel(pl.LightningModule):
  # Instantiate the model
  def __init__(self, learning_rate, tokenizer, model, hparams):
    super().__init__()
    self.tokenizer = tokenizer
    self.model = model
    self.learning_rate = learning_rate
    # self.freeze_encoder = freeze_encoder
    # self.freeze_embeds_ = freeze_embeds
    # self.hparams = hparams
    self.save_hyperparameters(hparams)

    if self.hparams.freeze_encoder:
      freeze_params(self.model.get_encoder())

    if self.hparams.freeze_embeds:
      self.freeze_embeds()
  
  def freeze_embeds(self):
    ''' freeze the positional embedding parameters of the model; adapted from finetune.py '''
    freeze_params(self.model.model.shared)
    for d in [self.model.model.encoder, self.model.model.decoder]:
      freeze_params(d.embed_positions)
      freeze_params(d.embed_tokens)

  # Do a forward pass through the model
  def forward(self, input_ids, **kwargs):
    return self.model(input_ids, **kwargs)
  
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
    return optimizer

  def training_step(self, batch, batch_idx):
    # Load the data into variables
    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]
    # Shift the decoder tokens right (but NOT the tgt_ids)
    decoder_input_ids = shift_tokens_right(tgt_ids, tokenizer.pad_token_id)

    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]
    # Create the loss function
    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    # Calculate the loss on the un-shifted tokens
    loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss':loss}

  def validation_step(self, batch, batch_idx):

    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]

    decoder_input_ids = shift_tokens_right(tgt_ids, tokenizer.pad_token_id)
    
    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]

    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    val_loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss': val_loss}
  
  # Method that generates text using the BartForConditionalGeneration's generate() method
  def generate_text(self, text, eval_beams, early_stopping = True, max_len = 1000):
    ''' Function to generate text '''
    generated_ids = self.model.generate(
        text["input_ids"],
        attention_mask=text["attention_mask"],
        use_cache=True,
        decoder_start_token_id = self.tokenizer.pad_token_id,
        num_beams= eval_beams,
        max_length = max_len,
        early_stopping = early_stopping
    )
    return [self.tokenizer.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in generated_ids]

  def training_epoch_end(self,outputs):
        #  the function is called after every epoch is completed

        # calculating average loss  
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()

        # calculating correect and total predictions
        # correct=sum([x["correct"] for  x in outputs])
        # total=sum([x["total"] for  x in outputs])

        # creating log dictionary
        # tensorboard_logs = {'loss': avg_loss,"Accuracy": correct/total}
        tensorboard_logs = {'loss': avg_loss}

        epoch_dictionary={
            # required
            'loss': avg_loss,
            
            # for logging purposes
            'log': tensorboard_logs}

        # return epoch_dictionary

def freeze_params(model):
  ''' Function that takes a model as input (or part of a model) and freezes the layers for faster training
      adapted from finetune.py '''
  for layer in model.parameters():
    layer.requires_grade = False


In [10]:
# Create the hparams dictionary to pass in the model
# I realise that this isn't really how this is meant to be used, but having this here reminds me that I can edit it when I need
hparams = argparse.Namespace()

hparams.freeze_encoder = True
hparams.freeze_embeds = True
hparams.eval_beams = 4

In [11]:
def shift_tokens_right(input_ids, pad_token_id):
  """ Shift input ids one token to the right, and wrap the last non pad token (usually <eos>).
      This is taken directly from modeling_bart.py
  """
  prev_output_tokens = input_ids.clone()
  index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
  prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
  prev_output_tokens[:, 1:] = input_ids[:, :-1]
  return prev_output_tokens

def encode_sentences(tokenizer, source_sentences, target_sentences, max_length=32, pad_to_max_length=True, return_tensors="pt"):
  ''' Function that tokenizes a sentence 
      Args: tokenizer - the BART tokenizer; source and target sentences are the source and target sentences
      Returns: Dictionary with keys: input_ids, attention_mask, target_ids
  '''

  input_ids = []
  attention_masks = []
  target_ids = []
  tokenized_sentences = {}

  for sentence in source_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim = 0)
  attention_masks = torch.cat(attention_masks, dim = 0)

  for sentence in target_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )
    # Shift the target ids to the right
    # shifted_target_ids = shift_tokens_right(encoded_dict['input_ids'], tokenizer.pad_token_id)
    target_ids.append(encoded_dict['input_ids'])

  target_ids = torch.cat(target_ids, dim = 0)
  

  batch = {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
      "labels": target_ids,
  }

  return batch


def noise_sentence(sentence_, percent_words, replacement_token = "<mask>"):
  '''
  Function that noises a sentence by adding <mask> tokens
  Args: sentence - the sentence to noise
        percent_words - the percent of words to replace with <mask> tokens; the number is rounded up using math.ceil
  Returns a noised sentence
  '''
  # Create a list item and copy
  sentence_ = sentence_.split(' ')
  sentence = sentence_.copy()
  
  num_words = math.ceil(len(sentence) * percent_words)
  
  # Create an array of tokens to sample from; don't include the last word as an option because in the case of lyrics
  # that word is often a rhyming word and plays an important role in song construction
  sample_tokens = set(np.arange(0, np.maximum(1, len(sentence)-1)))
  
  words_to_noise = random.sample(sample_tokens, num_words)
  
  # Swap out words, but not full stops
  for pos in words_to_noise:
      if sentence[pos] != '.':
          sentence[pos] = replacement_token
  
  # Remove redundant spaces
  sentence = re.sub(r' {2,5}', ' ', ' '.join(sentence))
  
  # Combine concurrent <mask> tokens into a single token; this just does two rounds of this; more could be done
  sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
  sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
  return sentence
  

# Load BART
Here we load the model. I used "bart-base" because I had memory issues using "bart-large". "bart-base" appears to load without the use_cache argument, which by necessity must be turned to "False" for "bart-large".

In [12]:
# Load the model
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base', add_prefix_space=True)

bart_model = BartForConditionalGeneration.from_pretrained(
    "facebook/bart-base")


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

In [13]:
# Load the data into the model for training
summary_data = SummaryDataModule(tokenizer, train, test, batch_size = 16)

# Load the model from a pre-saved checkpoint; alternatively use the code below to start training from scratch
# model = LitModel.load_from_checkpoint(root_dir + "checkpoint_files_3/5_epoch_11_29_ls.ckpt",
#                                       learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model, hparams = hparams)

model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model, hparams = hparams)

# Training the model with Pytorch Lightning
The below code utilises Pytorch Lightning's fantastic Trainer module that helps to control the training process. After creating a ModelCheckpoint object, the other options are fed into the Trainer module. I found that my colab crashed when I didn't explicitly set progress_bar_refresh_rate to something and I found that setting it to 500 seemed to work just fine.

In [14]:

checkpoint = ModelCheckpoint(dirpath=root_dir + 'checkpoint_files_3/')
trainer = pl.Trainer(gpus = 1,
                     max_epochs = 3,
                     min_epochs = 3,
                     auto_lr_find = False,
                     checkpoint_callback = checkpoint,
                     progress_bar_refresh_rate = 500)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:148: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f368f304c50>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f368f304c50>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=500)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the T

In [15]:
# Fit the instantiated model to the data
trainer.fit(model, summary_data)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M 
-------------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
557.682   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [16]:
# %load_ext tensorboard
# %tensorboard --logdir /content/lightning_logs/

In [17]:
# If you want to manually save a checkpoint, this works, although the model should automatically save (progressively better)
# checkpoints as it moves through the epochs
trainer.save_checkpoint(root_dir + "checkpoint_files_3/3_epoch_11_30_ls.ckpt")

# Generate Story from StoryLine

In [18]:
# def generate_lyrics(seed_line, num_lines, model_, noise_percent = 0.25, multiple_lines = False, max_line_history = 3):
#   ''' Function that generates lyrics based on previously generated lyrics 
#       Args: seed_line - a line to start off the machine
#             num_lines - the number of lines to generate
#             model_ - the model used to generate the text
#             multiple_lines - whether the model generates based on multiple previous lines or just the past line
#             max_line_history - the maximum number of previous lines used in the current input
#       Returns a list with num_lines of rap lines
#   '''
#   # Put the model on eval mode
#   model_.to(torch.device('cpu'))
#   model_.eval()
#   lyrics = []
#   lyrics.append(seed_line)
#   #skip noising for now
#   # prompt_line_tokens = tokenizer(noise_sentence(seed_line, 0.2), max_length = 600, return_tensors = "pt", truncation = True)
#   prompt_line_tokens = tokenizer(seed_line, max_length = 100, return_tensors = "pt", truncation = True)
#   # Loop through the number of lines generating a new line based on the old

#   line = [seed_line]
#   for i in range(num_lines):
#     # Print out the new line
#     print(line[0].strip())
#     lyrics.append(line[0])
#     line = model.generate_text(prompt_line_tokens, eval_beams = 4, max_len = 60000)
#     # This deals with an artefact in the training data that I had an issue cleaning
#     if line[0].find(":") != -1:
#       line[0] = re.sub(r'[A-Z]+: ', '', line[0])
#     # This allows the model to generate a new line conditioned on more than one line
#     if multiple_lines:
#       start_line = np.maximum(0, i - max_line_history)
#       end_line = i
#       prompt_line = ' '.join(lyrics[start_line:end_line]) # Going to end_line is fine because it is non-inclusive
#     else:
#       prompt_line = lyrics[i]
#     # prompt_line_tokens = tokenizer(noise_sentence(prompt_line, noise_percent), max_length = 32, return_tensors = "pt", truncation = True)
#     prompt_line_tokens = tokenizer(prompt_line, max_length = 32, return_tensors = "pt", truncation = True)

#   return lyrics

In [19]:
def generate_story(title_line, model_):
  # Put the model on eval mode
  model_.to(torch.device('cpu'))
  model_.eval()
  prompt_line_tokens = tokenizer(title_line, max_length = 100, return_tensors = "pt", truncation = True)
  out = model.generate_text(prompt_line_tokens, eval_beams = 4, early_stopping = False, max_len=100000)
  return out[0].strip()

In [20]:
df

,storytitle,storyline,story,titleLineConcat
0,David Drops the Weight,put try realized started weeks,David noticed he had put on a lot of weight re...,David Drops the Weight <EOT> put try realized ...
1,Frustration,tom angry wall tom regret,Tom had a very short temper.One day a guest ma...,Frustration <EOT> tom angry wall tom regret
2,Marcus Buys Khakis,business formal pair perfectly marcus,Marcus needed clothing for a business casual e...,Marcus Buys Khakis <EOT> business formal pair ...
3,Different Opinions,trailer needed much ways truck,Bobby thought Bill should buy a trailer and ha...,Different Opinions <EOT> trailer needed much w...
4,Overcoming shortcomings,pastor tried sing sundays delighted,John was a pastor with a very bad memory.He tr...,Overcoming shortcomings <EOT> pastor tried sin...
...,...,...,...,...
52660,Flavor,flavor tried get recipe recipe,The man liked the flavor.He tried to recreate ...,Flavor <EOT> flavor tried get recipe recipe
52661,After Death,trouble day found told week,"After my friend's dad's funeral, I got in trou...",After Death <EOT> trouble day found told week
52662,Janice breaks her wrist,janice legs working wrist wrist,Janice was out exercising for her big soccer g...,Janice breaks her wrist <EOT> janice legs work...
52663,Jamie marries for love,jamie married man marrying spent,Jamie is an american girl.Jamie wants to get m...,Jamie marries for love <EOT> jamie married man...


In [21]:
titleLineConcat = df['titleLineConcat'][0]
titleLineConcat

'David Drops the Weight <EOT> put try realized started weeks'

In [22]:
story = df['story'][0]
story

"David noticed he had put on a lot of weight recently.He examined his habits to try and figure out the reason.He realized he'd been eating too much fast food lately.He stopped going to burger places and started a vegetarian diet.After a few weeks, he started to feel much better."

In [23]:
story = generate_story(titleLineConcat, model)
story
#3 epoch
#Frustration
#jimmy told told told said
#David Drops the Weight
#dave overweight weight weight weight
#8 epoches
#David Drops the Weight
#lose wanted diet doctor doctor
#Frustration
#jim jim told told jim
#13 epoches
# Frustration
# tom angry wall tom tom
# David Drops the Weight
# pounds decided went month pounds

'David decided to put on a few pounds.He decided to try to lose weight.He realized that he was gaining a lot.He started to lose'

In [24]:
reduced_test = pd.read_csv(root_dir + 'test_predicted_storyline_15_ep.csv')
reduced_test

,storytitle,storyline,story,titleLineConcat,predicted_storylines,predicted_titleLineConcat
0,Crazed,cleaning seem went exterior sun,I was cleaning nonstop.I couldn't seem to get ...,Crazed <EOT> cleaning seem went exterior sun,kate door kate room kate,Crazed <EOT> kate door kate room kate
1,Thanksgiving Dinner.,house family thought good fine,Heather has decided to have Thanksgiving dinne...,Thanksgiving Dinner. <EOT> house family though...,thanksgiving turkey cooked ate well,Thanksgiving Dinner. <EOT> thanksgiving turkey...
2,On Sale,searching want saw bought saved,Rick was searching for a bicycle helmet.But he...,On Sale <EOT> searching want saw bought saved,went saw tried tried one,On Sale <EOT> went saw tried tried one
3,First Scraped Knee,eric ground saw mother washed,Eric was only 5 years old.He was playing outsi...,First Scraped Knee <EOT> eric ground saw mothe...,niece scraped knee scraped treat,First Scraped Knee <EOT> niece scraped knee sc...
4,tv,show writing youtube started got,Tom wanted to make a show.He started drawing a...,tv <EOT> show writing youtube started got,tv sudden tv tv tv,tv <EOT> tv sudden tv tv tv
...,...,...,...,...,...,...
256,Rear Ended,police man site trying subdue,I called the police after being rear ended.The...,Rear Ended <EOT> police man site trying subdue,tom tried right ended tom,Rear Ended <EOT> tom tried right ended tom
257,Bat,baseball made swung half larry,Larry tried out for the baseball team.He was s...,Bat <EOT> baseball made swung half larry,bat hit leg leg leg,Bat <EOT> bat hit leg leg leg
258,Her Mom,estranged seen contacts reacquaint mother,Ali was estranged from her birth mother.She ha...,Her Mom <EOT> estranged seen contacts reacquai...,mother year year mother mother,Her Mom <EOT> mother year year mother mother
259,Pat the Photographer,park saw took photo secretive,Pat went to the park to take photos.He saw a c...,Pat the Photographer <EOT> park saw took photo...,pat photographer started started started,Pat the Photographer <EOT> pat photographer st...


In [25]:
titleLineConcat = reduced_test['predicted_titleLineConcat'][0]
titleLineConcat

'Crazed <EOT> kate door kate room kate'

In [26]:
story = reduced_test['story'][0]
story

"I was cleaning nonstop.I couldn't seem to get anything clean enough.I went out and bought some cleaning supplies when I ran out.I went back home and started power washing the exterior.When I was done, the sun was going down."

In [27]:
story = generate_story(titleLineConcat, model)
story

'Kate was playing outside with her friends.A man knocked on the door.Kate was startled.The man ran out of the room.Kate had to'

In [28]:
#TODO: Figure out how to generate complete sentences
#maybe sep_token for sentences?
keep_index = []
generated_stories = []
for index, row in reduced_test.iterrows():
    out = generate_story(row['predicted_titleLineConcat'], model)
    if len(out.split('.')) >= 6:
      story = out.split('.')[:5]
      story = '.'.join(story) + '.'
      generated_stories.append(story)
      keep_index.append(index)
      print('title: ', row['storytitle'])
      print('origin_storyline: ', row['storyline'])
      print('origin_story: ', row['story'])
      print('generated_storylines: ', row['predicted_storylines'])
      print('generated_story: ', story)
      print()

title:  Drunk Dialing
origin_storyline:  tom decided started exes problems
origin_story:  Tom had problems with depression.He decided to drink one night.He went overboard and started calling people.He called some exes.It create a lot of problems in his life.
generated_storylines:  phone conversation man man apologized
generated_story:  The man hung up the phone.The woman continued to have a conversation.The man was drunk.The women confronted the man.They apologized.

title:  Scrambled Eggs
origin_storyline:  surprise usually morning woke missed
origin_story:  Aya woke early one morning to surprise her husband with a hot meal.He began work at 6AM, so usually she was only awake to say goodbye.But this morning, she made him a big hot dish of scrambled eggs.Then she woke him and smiled as he dove in - until he grimaced.Aya had missed a large chunk of eggshell, and it had cut her husband!
generated_storylines:  eggs eggs eggs scrambled eggs
generated_story:  The man made some eggs.He scramb

In [29]:
reduced_test = reduced_test.iloc[keep_index]

In [30]:
reduced_test['predicted_story'] = generated_stories

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
for index, row in reduced_test.iterrows():
    out = row['predicted_story']
    if len(out.split('.')) != 6:
      print(out)

In [32]:
reduced_test.reset_index(drop=True, inplace=True)

In [33]:
reduced_test

,storytitle,storyline,story,titleLineConcat,predicted_storylines,predicted_titleLineConcat,predicted_story
0,Drunk Dialing,tom decided started exes problems,Tom had problems with depression.He decided to...,Drunk Dialing <EOT> tom decided started exes p...,phone conversation man man apologized,Drunk Dialing <EOT> phone conversation man man...,The man hung up the phone.The woman continued ...
1,Scrambled Eggs,surprise usually morning woke missed,Aya woke early one morning to surprise her hus...,Scrambled Eggs <EOT> surprise usually morning ...,eggs eggs eggs scrambled eggs,Scrambled Eggs <EOT> eggs eggs eggs scrambled ...,The man made some eggs.He scrambled the eggs.T...
2,Trick or Treating,loved moved treating try candy,Tim loved Halloween.He moved into a nice neigh...,Trick or Treating <EOT> loved moved treating t...,halloween dressed year dressed year,Trick or Treating <EOT> halloween dressed year...,It was Halloween.Everyone was dressed up.This ...
3,Breakfast,suzanne family time served time,Suzanne was in the kitchen making breakfast.He...,Breakfast <EOT> suzanne family time served time,hungry decided ingredients followed hunger,Breakfast <EOT> hungry decided ingredients fol...,Tom woke up hungry.He decided to make breakfas...
4,The Internet,trouble weeks seem turned uses,Lynn was having a lot of trouble with her inte...,The Internet <EOT> trouble weeks seem turned uses,internet went called fixed internet,The Internet <EOT> internet went called fixed ...,The internet was broken.I went to the internet...
5,Wrong Party,tom sure went party one,Tom was invited to a party.He wasn't sure abou...,Wrong Party <EOT> tom sure went party one,tim birthday tim wrong tim,Wrong Party <EOT> tim birthday tim wrong tim,Tim was invited to a birthday party.It was his...
6,Pets,son needed home every happy,My son had always wanted a pet.I found two cat...,Pets <EOT> son needed home every happy,pet went found took named,Pets <EOT> pet went found took named,The man wanted a pet.He went to the pet store....
7,Too sweet,get would took sweet could,Allie wanted to get some candy.But she was not...,Too sweet <EOT> get would took sweet could,sweet drink sweet drink stomach,Too sweet <EOT> sweet drink sweet drink stomach,The man had a sweet tooth.He decided to drink ...
8,Boring Movie,romance really tim whining movie,Tim hated romance movies.His girlfriend really...,Boring Movie <EOT> romance really tim whining ...,tom watching tom wound tom,Boring Movie <EOT> tom watching tom wound tom,Tom was at the movies.He was watching a horror...
9,Stolen Car,tim parked missing report unfortunately,Tim was visiting friends.He parked in a bad ne...,Stolen Car <EOT> tim parked missing report unf...,car stolen tom police towed,Stolen Car <EOT> car stolen tom police towed,Tom bought a car.The car was stolen.Tom was an...


In [34]:
reduced_test.to_csv(root_dir+'test_predicted_story_3_ep.csv', encoding='utf-8', index=False)

# Evaluate BLEU Score

In [38]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
score_list = []
for index, row in reduced_test.iterrows():
    gt = row['story']
    pred = row['predicted_story']
    refs = []
    for s in gt.split('.')[:5]:
      refs.append(s.split())
    # print(refs)
    cans = []
    for s in pred.split('.')[:5]:
      can = s.split()
      cans.extend(can)
    score = sentence_bleu(refs, cans)
    score_list.append(score)

print('sentence bleu score is: ', np.mean(score_list))

sentence bleu score is:  0.4354171330439157


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [41]:
#output notebook as pdf
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('bart_storyline_2_story.ipynb')

File ‘colab_pdf.py’ already there; not retrieving.





Extracting templates from packages: 100%
[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/bart_storyline_2_story.ipynb to pdf
[NbConvertApp] Writing 103589 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 88579 bytes to /content/drive/My Drive/bart_storyline_2_story.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'File ready to be Downloaded and Saved to Drive'